In [1]:
import requests
import calendar
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import os
import re
import time
import numpy as np

### Example

In [2]:
email = 'xianlin.ding@sciencespo.fr'
password = '099115Keep12??'
path = os.getcwd()
path = re.sub('Code','Result',path)

In [3]:
# go into the Eruopresse website
url = "https://acces-distant.sciencespo.fr/fork?https://nouveau.europresse.com/access/ip/default.aspx?un=politique2T_1"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.find_element(By.ID, 'username').send_keys(email)
driver.find_element(By.ID, 'password').send_keys(password)
time.sleep(3)
driver.find_element(By.CLASS_NAME, 'btn-submit').click()

<ipython-input-3-4fdf21dd8406>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
# choose the criteria of searching
driver.find_element(By.CLASS_NAME, 'lnk-text').click()
time.sleep(3)
driver.find_element(By.ID, 'specific-sources-rd').click()
time.sleep(5)
driver.find_element(By.ID, 'sf_247').click() # le monde

In [5]:
# period: 1/oct/2022 - 1/dec/2022
Select(driver.find_element(By.ID, 'DateFilter_DateRange')).select_by_value('10')

Select(driver.find_elements(By.CLASS_NAME, 'year')[1]).select_by_value('2013')
Select(driver.find_elements(By.CLASS_NAME, 'year')[2]).select_by_value('2013')

In [6]:
Select(driver.find_elements(By.CLASS_NAME, 'month')[1]).select_by_value('1')
Select(driver.find_elements(By.CLASS_NAME, 'month')[2]).select_by_value('1')

In [7]:
Select(driver.find_elements(By.CLASS_NAME, 'day')[1]).select_by_value('1')
Select(driver.find_elements(By.CLASS_NAME, 'day')[2]).select_by_value('31')

In [6]:
driver.find_element(By.ID, 'btnSearch').click()

In [12]:
# get web contents
web_content = driver.page_source
obj = BeautifulSoup(web_content,'lxml').body
obj = obj.aside.div.div.div.div.div
obj = obj.find_all(name='span', attrs={"class":"resultOperations-count"})[0]
obj = int(re.sub(r'\xa0','',obj.contents[0]))
obj

In [8]:
driver.close()

### Function

In [13]:
def scrape(journal_code, year_i, month_i):
    
    email = 'xianlin.ding@sciencespo.fr'
    password = '099115Keep12??'
    
    # go into the Eruopresse website
    url = "https://acces-distant.sciencespo.fr/fork?https://nouveau.europresse.com/access/ip/default.aspx?un=politique2T_1"
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.find_element(By.ID, 'username').send_keys(email)
    driver.find_element(By.ID, 'password').send_keys(password)
    time.sleep(3)
    driver.find_element(By.CLASS_NAME, 'btn-submit').click()
    
    # choose the criteria of searching
    driver.find_element(By.CLASS_NAME, 'lnk-text').click()
    time.sleep(3)
    driver.find_element(By.ID, 'specific-sources-rd').click()
    time.sleep(5)
            
    # choose the journal
    try:
        driver.find_element(By.ID, journal_code).click()
    except:
        time.sleep(20)
        driver.find_element(By.ID, journal_code).click()
                
    # choose the searching period
    day_start = 1
    day_end = calendar.monthrange(year_i,month_i)[1]
    
    Select(driver.find_element(By.ID, 'DateFilter_DateRange')).select_by_value('10')

    Select(driver.find_elements(By.CLASS_NAME, 'year')[1]).select_by_value(str(year_i))
    time.sleep(2)
    Select(driver.find_elements(By.CLASS_NAME, 'year')[2]).select_by_value(str(year_i))
    time.sleep(2)
    
    Select(driver.find_elements(By.CLASS_NAME, 'month')[1]).select_by_value(str(month_i))
    time.sleep(2)
    Select(driver.find_elements(By.CLASS_NAME, 'month')[2]).select_by_value(str(month_i))      
    time.sleep(2)
    
    Select(driver.find_elements(By.CLASS_NAME, 'day')[1]).select_by_value(str(day_start))
    time.sleep(2)
    Select(driver.find_elements(By.CLASS_NAME, 'day')[2]).select_by_value(str(day_end))
    time.sleep(2)
    
    driver.find_element(By.ID, 'btnSearch').click()
    time.sleep(5)
    
    # get web contents
    web_content = driver.page_source
    obj = BeautifulSoup(web_content,'lxml').body
    obj = obj.aside.div.div.div.div.div
    obj = obj.find_all(name='span', attrs={"class":"resultOperations-count"})[0]
    obj = int(re.sub(r'\xa0','',obj.contents[0]))
    
    driver.close()
    
    return obj


In [14]:
# define the function

def scrape_all_articles(journal_code, year_start, year_end):
    
    year = list(range(year_start,year_end+1))
    month = list(range(1,13))
    
    info = pd.DataFrame(columns = ['year', 'month', 'number'])
    i=0
    for year_i in year:
        for month_i in month:
            
            while True:
                try:
                    obj = scrape(journal_code, year_i, month_i)
                    break
                except:
                    print('Maximum access times reach, sleep 5 minutes.')
                    time.sleep(300)    
            
            info.loc[i] = [year_i, month_i, obj]
            print('Year ', year_i,', Month ', month_i, ', ', obj, ' finished.')
            i=i+1
            
            
    return info


### Le Figaro

In [15]:
journal_code = 'sf_246'
info = scrape_all_articles(journal_code, 2013, 2022)

<ipython-input-13-d39ba76301df>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Year  2013 , Month  1 ,  5825  finished.
Year  2013 , Month  2 ,  5579  finished.
Year  2013 , Month  3 ,  5737  finished.
Year  2013 , Month  4 ,  5449  finished.
Maximum access times reach, sleep 5 minutes.
Year  2013 , Month  5 ,  4968  finished.
Year  2013 , Month  6 ,  5207  finished.
Year  2013 , Month  7 ,  3918  finished.
Year  2013 , Month  8 ,  2942  finished.
Year  2013 , Month  9 ,  4528  finished.
Year  2013 , Month  10 ,  4850  finished.
Year  2013 , Month  11 ,  4576  finished.
Maximum access times reach, sleep 5 minutes.
Year  2013 , Month  12 ,  4429  finished.
Year  2014 , Month  1 ,  4586  finished.
Year  2014 , Month  2 ,  4325  finished.
Maximum access times reach, sleep 5 minutes.
Year  2014 , Month  3 ,  4916  finished.
Year  2014 , Month  4 ,  4567  finished.
Year  2014 , Month  5 ,  4611  finished.
Year  2014 , Month  6 ,  4484  finished.
Year  2014 , Month  7 ,  3966  finished.
Year  2014 , Month  8 ,  3187  finished.
Year  2014 , Month  9 ,  4807  finished.
Y

In [16]:
path = os.getcwd()
path = re.sub('Code','Result',path)
info.to_csv(path + '\\data_raw_final\\'+ 'Le Figaro_All' + '.csv')

### Le Parisien

In [17]:
journal_code = 'sf_257'
info = scrape_all_articles(journal_code, 2013, 2022)

<ipython-input-13-d39ba76301df>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Year  2013 , Month  1 ,  11031  finished.
Year  2013 , Month  2 ,  10472  finished.
Year  2013 , Month  3 ,  10138  finished.
Year  2013 , Month  4 ,  10220  finished.
Year  2013 , Month  5 ,  9334  finished.
Year  2013 , Month  6 ,  9432  finished.
Year  2013 , Month  7 ,  7933  finished.
Year  2013 , Month  8 ,  6852  finished.
Year  2013 , Month  9 ,  9530  finished.
Year  2013 , Month  10 ,  9984  finished.
Year  2013 , Month  11 ,  10181  finished.
Year  2013 , Month  12 ,  9313  finished.
Year  2014 , Month  1 ,  10110  finished.
Maximum access times reach, sleep 5 minutes.
Maximum access times reach, sleep 5 minutes.
Year  2014 , Month  2 ,  10568  finished.
Year  2014 , Month  3 ,  11902  finished.
Year  2014 , Month  4 ,  10221  finished.
Year  2014 , Month  5 ,  10300  finished.
Year  2014 , Month  6 ,  9897  finished.
Year  2014 , Month  7 ,  9709  finished.
Year  2014 , Month  8 ,  8029  finished.
Year  2014 , Month  9 ,  10624  finished.
Year  2014 , Month  10 ,  11019  fi

In [18]:
path = os.getcwd()
path = re.sub('Code','Result',path)
info.to_csv(path + '\\data_raw_final\\'+ 'Le Parisien_All' + '.csv')

### La Croix

In [19]:
journal_code = 'sf_243'
info = scrape_all_articles(journal_code, 2013, 2022)

<ipython-input-13-d39ba76301df>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Maximum access times reach, sleep 5 minutes.
Year  2013 , Month  1 ,  2266  finished.
Year  2013 , Month  2 ,  1821  finished.
Year  2013 , Month  3 ,  1838  finished.
Year  2013 , Month  4 ,  1932  finished.
Year  2013 , Month  5 ,  1699  finished.
Year  2013 , Month  6 ,  1944  finished.
Year  2013 , Month  7 ,  1952  finished.
Year  2013 , Month  8 ,  1767  finished.
Year  2013 , Month  9 ,  1858  finished.
Year  2013 , Month  10 ,  2010  finished.
Year  2013 , Month  11 ,  1874  finished.
Year  2013 , Month  12 ,  1786  finished.
Year  2014 , Month  1 ,  2129  finished.
Year  2014 , Month  2 ,  1790  finished.
Year  2014 , Month  3 ,  1998  finished.
Year  2014 , Month  4 ,  1826  finished.
Year  2014 , Month  5 ,  1850  finished.
Maximum access times reach, sleep 5 minutes.
Year  2014 , Month  6 ,  1926  finished.
Year  2014 , Month  7 ,  1904  finished.
Year  2014 , Month  8 ,  1795  finished.
Year  2014 , Month  9 ,  2032  finished.
Year  2014 , Month  10 ,  2088  finished.
Year

In [20]:
path = os.getcwd()
path = re.sub('Code','Result',path)
info.to_csv(path + '\\data_raw_final\\'+ 'La Croix_All' + '.csv')

### Libération

In [21]:
journal_code = 'sf_252'
info = scrape_all_articles(journal_code, 2013, 2022)

<ipython-input-13-d39ba76301df>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Year  2013 , Month  1 ,  2292  finished.
Year  2013 , Month  2 ,  2184  finished.
Year  2013 , Month  3 ,  2161  finished.
Year  2013 , Month  4 ,  2221  finished.
Year  2013 , Month  5 ,  2146  finished.
Year  2013 , Month  6 ,  2251  finished.
Year  2013 , Month  7 ,  2288  finished.
Year  2013 , Month  8 ,  1981  finished.
Year  2013 , Month  9 ,  2137  finished.
Year  2013 , Month  10 ,  2430  finished.
Year  2013 , Month  11 ,  2294  finished.
Year  2013 , Month  12 ,  2178  finished.
Year  2014 , Month  1 ,  2205  finished.
Maximum access times reach, sleep 5 minutes.


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:00<00:00, 43.3MB/s]


Year  2014 , Month  2 ,  2056  finished.
Maximum access times reach, sleep 5 minutes.
Year  2014 , Month  3 ,  2281  finished.
Year  2014 , Month  4 ,  2118  finished.
Year  2014 , Month  5 ,  2330  finished.
Year  2014 , Month  6 ,  2251  finished.
Year  2014 , Month  7 ,  2222  finished.
Year  2014 , Month  8 ,  1948  finished.
Year  2014 , Month  9 ,  2196  finished.
Year  2014 , Month  10 ,  2197  finished.
Year  2014 , Month  11 ,  1994  finished.
Year  2014 , Month  12 ,  2052  finished.
Year  2015 , Month  1 ,  1819  finished.
Year  2015 , Month  2 ,  1686  finished.
Year  2015 , Month  3 ,  1772  finished.
Year  2015 , Month  4 ,  1692  finished.
Year  2015 , Month  5 ,  1779  finished.
Year  2015 , Month  6 ,  1708  finished.
Year  2015 , Month  7 ,  1593  finished.
Year  2015 , Month  8 ,  1456  finished.
Maximum access times reach, sleep 5 minutes.
Year  2015 , Month  9 ,  1606  finished.
Year  2015 , Month  10 ,  1725  finished.
Year  2015 , Month  11 ,  1573  finished.
Yea

In [22]:
path = os.getcwd()
path = re.sub('Code','Result',path)
info.to_csv(path + '\\data_raw_final\\'+ 'Libération_All' + '.csv')

### Les Echos

In [23]:
journal_code = 'sf_251'
info = scrape_all_articles(journal_code, 2013, 2022)

<ipython-input-13-d39ba76301df>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Year  2013 , Month  1 ,  3503  finished.
Year  2013 , Month  2 ,  3349  finished.
Year  2013 , Month  3 ,  3600  finished.
Maximum access times reach, sleep 5 minutes.
Year  2013 , Month  4 ,  3470  finished.
Year  2013 , Month  5 ,  3129  finished.
Year  2013 , Month  6 ,  3567  finished.
Year  2013 , Month  7 ,  3350  finished.
Year  2013 , Month  8 ,  2500  finished.
Year  2013 , Month  9 ,  3426  finished.
Year  2013 , Month  10 ,  3878  finished.
Year  2013 , Month  11 ,  3171  finished.
Maximum access times reach, sleep 5 minutes.
Year  2013 , Month  12 ,  3174  finished.
Year  2014 , Month  1 ,  3489  finished.
Year  2014 , Month  2 ,  3341  finished.
Year  2014 , Month  3 ,  3583  finished.
Year  2014 , Month  4 ,  3285  finished.
Year  2014 , Month  5 ,  3107  finished.
Year  2014 , Month  6 ,  3217  finished.
Year  2014 , Month  7 ,  2944  finished.
Year  2014 , Month  8 ,  2249  finished.
Year  2014 , Month  9 ,  3606  finished.
Year  2014 , Month  10 ,  3713  finished.
Year

In [24]:
path = os.getcwd()
path = re.sub('Code','Result',path)
info.to_csv(path + '\\data_raw_final\\'+ 'Les Echos_All' + '.csv')

### Le Monde

In [25]:
journal_code = 'sf_247'
info = scrape_all_articles(journal_code, 2013, 2022)

<ipython-input-13-d39ba76301df>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Year  2013 , Month  1 ,  3120  finished.
Year  2013 , Month  2 ,  3145  finished.
Year  2013 , Month  3 ,  3512  finished.
Year  2013 , Month  4 ,  3196  finished.
Year  2013 , Month  5 ,  3451  finished.
Year  2013 , Month  6 ,  3291  finished.
Year  2013 , Month  7 ,  2997  finished.
Year  2013 , Month  8 ,  2417  finished.
Year  2013 , Month  9 ,  3078  finished.
Year  2013 , Month  10 ,  3195  finished.
Year  2013 , Month  11 ,  3158  finished.
Year  2013 , Month  12 ,  2862  finished.
Year  2014 , Month  1 ,  2862  finished.
Year  2014 , Month  2 ,  2826  finished.
Maximum access times reach, sleep 5 minutes.
Year  2014 , Month  3 ,  3255  finished.
Year  2014 , Month  4 ,  3589  finished.
Year  2014 , Month  5 ,  5084  finished.
Year  2014 , Month  6 ,  4948  finished.
Year  2014 , Month  7 ,  4799  finished.
Year  2014 , Month  8 ,  4022  finished.
Year  2014 , Month  9 ,  5924  finished.
Year  2014 , Month  10 ,  8593  finished.
Year  2014 , Month  11 ,  7879  finished.
Year  2

In [26]:
path = os.getcwd()
path = re.sub('Code','Result',path)
info.to_csv(path + '\\data_raw_final\\'+ 'Le Monde_All' + '.csv')